In [ ]:
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

Downloading...
From (original): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf
From (redirected): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf&confirm=t&uuid=55e6c7bd-96cf-4118-b59d-dd65e7d89bd8
To: /content/mpsis_setup.py
100%|██████████| 12.1k/12.1k [00:00<00:00, 5.90MB/s]


In [ ]:
def generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng):
    lat = random.uniform(sw_lat, ne_lat)
    lng = random.uniform(sw_lng, ne_lng)
    return lat, lng

def snap_to_road(lat, lng, api_key):
    url = f'https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
        snapped_point = data['snappedPoints'][0]['location']
        return snapped_point['latitude'], snapped_point['longitude']
    return None

def get_road_distance_time(origin, destination, api_key):
    origin_str = f'{origin[0]}, {origin[1]}'
    destination_str = f'{destination[0]}, {destination[1]}'
    departure_time = int(time.time()) + 24 * 3600
    url = (
        f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin_str}'
        f'&destinations={destination_str}&mode=driving&departure_time={departure_time}&key={api_key}'
    )
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        element = data['rows'][0]['elements'][0]
        if element['status'] == 'OK':
            distance_text = element['distance']['text']
            distance = float(distance_text.replace(' km', '').replace(',', ''))

            duration_text = element['duration']['text']
            duration_parts = duration_text.split()
            duration = int(duration_parts[0])
            if "hour" in duration_text:
                duration = int(duration_parts[0]) * 60 + int(duration_parts[2])

            return distance, duration
        else:
            return None, None
    else:
        return None, None

def find_center(sw_lat, sw_lng, ne_lat, ne_lng):
    center_lat = (sw_lat + ne_lat) / 2
    center_lng = (sw_lng + ne_lng) / 2
    return center_lat, center_lng

def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

def crop_matrix(data, N):
    return [row[:N] for row in data[:N]]

import numpy as np

def generate_demand_list(n, m, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if m >= n:
        raise ValueError("M must be less than N.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")

    random_values = np.random.rand(n - m - 1)  # n - (m + 1) non-zero elements
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    zeros_list = [0] * m

    result = list(rounded_values) + zeros_list

    random.shuffle(result)

    result = [-1] + result

    return result
def replace_zeros_with_sum(input_list, target_sum):
    zero_positions = [i for i, value in enumerate(input_list) if value == 0]
    num_zeros = len(zero_positions)

    result = input_list.copy()

    if num_zeros == 0:
        raise ValueError("There are no zeros to replace in the list.")

    random_values = np.random.rand(num_zeros)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    for i, pos in enumerate(zero_positions):
        result[pos] = rounded_values[i]

    return result

def create_path_as_list(pairs_list):
    wanted_length = len(pairs_list)
    path = ''
    current_pair = next(pair for pair in pairs_list if pair[0] == 0)
    path += str(current_pair[0])

    next_start = current_pair[1]

    pairs_list.remove(current_pair)

    while len(path) < wanted_length:
        path += str(next_start)
        current_pair = next(pair for pair in pairs_list if pair[0] == next_start)
        pairs_list.remove(current_pair)
        next_start = current_pair[1]

    return path

def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def repetitions(list1, list2):
  common_elements = set(list1) & set(list2)
  return (2*len(common_elements))/len(list1+list2)

def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [ ]:
def cvrp(number_of_clients, number_of_vehicles, cost_buffor, demand_of_client, file_idx, problem_idx):
    try:
        problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

        cost_shiffted = crop_matrix(cost_buffor, number_of_clients + 1)
        capacity_of_vehicle = 100
        x = [[[pulp.LpVariable(f"x{i}_{j},{k}", cat="Binary") if i != j else None
              for k in range(number_of_vehicles)]
              for j in range(number_of_clients + 1)]
            for i in range(number_of_clients + 1)]

        problem += pulp.lpSum(cost_shiffted[i][j] * x[i][j][k] if i != j else 0
                              for k in range(number_of_vehicles)
                              for j in range(number_of_clients + 1)
                              for i in range(number_of_clients + 1))

        for j in range(1, number_of_clients + 1):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                  for i in range(number_of_clients + 1)
                                  for k in range(number_of_vehicles)) == 1

        for k in range(number_of_vehicles):
            problem += pulp.lpSum(x[0][j][k] for j in range(1, number_of_clients + 1)) == 1
            problem += pulp.lpSum(x[i][0][k] for i in range(1, number_of_clients + 1)) == 1

        for k in range(number_of_vehicles):
            for j in range(number_of_clients + 1):
                problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                      for i in range(number_of_clients + 1)) - \
                          pulp.lpSum(x[j][i][k] for i in range(number_of_clients + 1)) == 0

        for k in range(number_of_vehicles):
            problem += pulp.lpSum(demand_of_client[j] * x[i][j][k] if i != j else 0
                                  for i in range(number_of_clients + 1)
                                  for j in range(1, number_of_clients + 1)) <= capacity_of_vehicle

        u = pulp.LpVariable.dicts("u", range(1, number_of_clients + 1), 0, number_of_clients, cat="Continuous")
        for i in range(1, number_of_clients + 1):
            for j in range(1, number_of_clients + 1):
                if i != j:
                    for k in range(number_of_vehicles):
                        problem += u[i] - u[j] + (number_of_clients * x[i][j][k]) <= number_of_clients - 1

        problem.solve()

        all_uesd_edges = []
        paths = [[] for _ in range(number_of_vehicles)]

        for k in range(number_of_vehicles):
            for i in range(number_of_clients + 1):
                for j in range(number_of_clients + 1):
                    if i != j and pulp.value(x[i][j][k]) == 1:
                        all_uesd_edges.append((i, j))
                        paths[k].append((i, j))

        result_paths = [create_path_as_list(lst.copy()) for lst in paths]

        return pulp.value(problem.objective), result_paths, all_uesd_edges
    except Exception as e:
      print(f"error for delivery stops {number_of_clients} and vehicles {number_of_vehicles}")

      values_to_reload.append((number_of_clients, number_of_vehicles, file_idx, problem_idx))

      return None, None, None

In [ ]:
def cvrp_error(number_of_clients, number_of_vehicles, cost_buffor, demand_of_client, file_idx):
    try:
        problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

        cost_shiffted = crop_matrix(cost_buffor, number_of_clients + 1)
        capacity_of_vehicle = 100
        x = [[[pulp.LpVariable(f"x{i}_{j},{k}", cat="Binary") if i != j else None
              for k in range(number_of_vehicles)]
              for j in range(number_of_clients + 1)]
            for i in range(number_of_clients + 1)]

        problem += pulp.lpSum(cost_shiffted[i][j] * x[i][j][k] if i != j else 0
                              for k in range(number_of_vehicles)
                              for j in range(number_of_clients + 1)
                              for i in range(number_of_clients + 1))

        for j in range(1, number_of_clients + 1):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                  for i in range(number_of_clients + 1)
                                  for k in range(number_of_vehicles)) == 1

        for k in range(number_of_vehicles):
            problem += pulp.lpSum(x[0][j][k] for j in range(1, number_of_clients + 1)) == 1
            problem += pulp.lpSum(x[i][0][k] for i in range(1, number_of_clients + 1)) == 1

        for k in range(number_of_vehicles):
            for j in range(number_of_clients + 1):
                problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                      for i in range(number_of_clients + 1)) - \
                          pulp.lpSum(x[j][i][k] for i in range(number_of_clients + 1)) == 0

        for k in range(number_of_vehicles):
            problem += pulp.lpSum(demand_of_client[j] * x[i][j][k] if i != j else 0
                                  for i in range(number_of_clients + 1)
                                  for j in range(1, number_of_clients + 1)) <= capacity_of_vehicle

        u = pulp.LpVariable.dicts("u", range(1, number_of_clients + 1), 0, number_of_clients, cat="Continuous")
        for i in range(1, number_of_clients + 1):
            for j in range(1, number_of_clients + 1):
                if i != j:
                    for k in range(number_of_vehicles):
                        problem += u[i] - u[j] + (number_of_clients * x[i][j][k]) <= number_of_clients - 1


        problem.solve()

        all_uesd_edges = []
        paths = [[] for _ in range(number_of_vehicles)]

        for k in range(number_of_vehicles):
            for i in range(number_of_clients + 1):
                for j in range(number_of_clients + 1):
                    if i != j and pulp.value(x[i][j][k]) == 1:
                        all_uesd_edges.append((i, j))
                        paths[k].append((i, j))

        result_paths = [create_path_as_list(lst.copy()) for lst in paths]

        return pulp.value(problem.objective), result_paths, all_uesd_edges
    except Exception as e:
        print(f"ONE MORE TIME ERROR {e}")
        return None, None, None

In [ ]:
def create_cost_matrixes(df):
    real_distance_columns = [col for col in df.columns if col.startswith('Road Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_raod = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Road Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_raod[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Time to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_time = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Time to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_time[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Real Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_real = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Real Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_real[i][j] = float(distance)

    return cost_raod, cost_time, cost_real

### CRACOW

In [ ]:
print(files)
# files = ['30_clients_cracow_1', '30_clients_cracow_2' #, '30_clients_cracow_3.csv','30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']

['30_clients_cracow_1', '30_clients_cracow_2', '30_clients_cracow_3.csv', '30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']


In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(2)]
all_paths_road = [[[] for _ in range(3)] for _ in range(2)]
all_paths_time = [[[] for _ in range(3)] for _ in range(2)]
all_paths_real = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(2)]

values_to_reload = []
vechicles = [1,2,3]
clients = [10,20]

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      demand_of_client = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)

      obj_road, paths_road, edges_road = cvrp(number_of_clients, number_of_vehicles, cost_R, demand_of_client, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = cvrp(number_of_clients, number_of_vehicles, cost_T, demand_of_client, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = cvrp(number_of_clients, number_of_vehicles, cost_G, demand_of_client, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

############# calculated_file 30_clients_cracow_1
error for delivery stops 20 and vehicles 2
############# calculated_file 30_clients_cracow_2
############# calculated_file 30_clients_cracow_3.csv
############# calculated_file 30_clients_cracow_4.csv
############# calculated_file 30_clients_cracow_5.csv
error for delivery stops 20 and vehicles 3
############# calculated_file 30_clients_cracow_6.csv
############# calculated_file 30_clients_cracow_7.csv
error for delivery stops 20 and vehicles 3
############# calculated_file 30_clients_cracow_8.csv
error for delivery stops 20 and vehicles 2
error for delivery stops 20 and vehicles 2
############# calculated_file 30_clients_cracow_9.csv
error for delivery stops 20 and vehicles 3
############# calculated_file 30_clients_cracow_10.csv
error for delivery stops 20 and vehicles 1
############# calculated_file 30_clients_cracow_11.csv
############# calculated_file 30_clients_cracow_12.csv


In [ ]:
percent = 0.7

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)
    deamnd = generate_list(number_of_clients + 1, percent * capacity_of_vehicle * number_of_vehicles)

    obj_1, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_road, deamnd, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_time, deamnd, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_real, deamnd, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(2):

      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))



In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(2)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_G = [[[] for _ in range(3)] for _ in range(2)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_T = [[[] for _ in range(3)] for _ in range(2)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_edges = [[[] for _ in range(3)] for _ in range(2)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_G_edges = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

G_T_edges = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_paths = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths = [[[] for _ in range(3)] for _ in range(2)]


for vechicles_idx in range(3):
  for clients_idx in range(2):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")

### EUROPE

In [ ]:
output_dir = '/content/drive/MyDrive/MAGISTERKA/cost_function_matrix'
files = [f for f in os.listdir(output_dir) if f.startswith("30_clients_europe")]
print(files)
# files = ['30_clients_cracow_1', '30_clients_cracow_2']#, '30_clients_cracow_3.csv','30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']

In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(2)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(2)]
all_paths_road = [[[] for _ in range(3)] for _ in range(2)]
all_paths_time = [[[] for _ in range(3)] for _ in range(2)]
all_paths_real = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(2)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(2)]

values_to_reload = []
vechicles = [1,2,3]
clients = [10,18]

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      demand_of_client = generate_list(number_of_clients + 1, 0.6 * capacity_of_vehicle * number_of_vehicles)

      obj_road, paths_road, edges_road = cvrp(number_of_clients, number_of_vehicles, cost_R, demand_of_client, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = cvrp(number_of_clients, number_of_vehicles, cost_T, demand_of_client, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = cvrp(number_of_clients, number_of_vehicles, cost_G, demand_of_client, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

In [ ]:
percent = 0.7

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)
    deamnd = generate_list(number_of_clients + 1, percent * capacity_of_vehicle * number_of_vehicles)

    obj_1, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_road, deamnd, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_time, deamnd, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_real, deamnd, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew

In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(2)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(2):

      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))



In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(2)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_G = [[[] for _ in range(3)] for _ in range(2)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(2)]

AVG_T = [[[] for _ in range(3)] for _ in range(2)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_edges = [[[] for _ in range(3)] for _ in range(2)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_G_edges = [[[] for _ in range(3)] for _ in range(2)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

G_T_edges = [[[] for _ in range(3)] for _ in range(2)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(2)]

R_T_paths = [[[] for _ in range(3)] for _ in range(2)]
R_G_paths = [[[] for _ in range(3)] for _ in range(2)]
G_T_paths = [[[] for _ in range(3)] for _ in range(2)]


for vechicles_idx in range(3):
  for clients_idx in range(2):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")

### RESULTS

In [ ]:
for client in clients:
  for vechicle in vechicles:
    df.loc[iterator] = [topology, client, vechicle,
                        str(AVG_R[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_R_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_G[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_G_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_T[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_T_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(R_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(R_G_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_G_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(G_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(G_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        R_T_paths[clients.index(client)][vechicles.index(vechicle)], R_G_paths[clients.index(client)][vechicles.index(vechicle)], G_T_paths[clients.index(client)][vechicles.index(vechicle)]
                        ]
    iterator += 1


In [ ]:
| Topology | No. clients | No. vehicles | Road distance matrix | Geographical distance matrix | Time distance matrix |
|----------|-------------|--------------|----------------------|-----------------------------|----------------------|
| Krakow   | 10          | 1            | 41.305 ± 4.987       | 18.563 ± 1.079              | 92.8 ± 9.377         |
| Krakow   | 10          | 2            | 44.47 ± 5.034        | 20.824 ± 1.111              | 103.45 ± 9.602       |
| Krakow   | 10          | 3            | 48.345 ± 4.931       | 23.551 ± 1.322              | 114.95 ± 9.551       |
| Krakow   | 15          | 1            | 53.764 ± 3.758       | 21.519 ± 0.915              | 122.05 ± 7.917       |
| Krakow   | 15          | 2            | 56.704 ± 3.775       | 23.588 ± 0.95               | 132.1 ± 8.019        |
| Krakow   | 15          | 3            | 60.489 ± 4.02        | 25.966 ± 1.224              | 143.3 ± 8.419        |
| Krakow   | 20          | 1            | 59.37 ± 2.707        | 24.025 ± 0.835              | 138.7 ± 5.968        |
| Krakow   | 20          | 2            | 62.24 ± 2.705        | 25.989 ± 0.923              | 148.1 ± 6.097        |
| Krakow   | 20          | 3            | 66.09 ± 2.734        | 28.304 ± 1.11               | 159.35 ± 6.186       |
| Europe   | 10          | 1            | 3261.11 ± 203.988    | 2426.369 ± 143.128          | 2279.55 ± 142.798    |
| Europe   | 10          | 2            | 3711.675 ± 251.796   | 2756.555 ± 172.838          | 2563.0 ± 174.09      |
| Europe   | 10          | 3            | 4219.59 ± 288.189    | 3136.775 ± 202.357          | 2885.8 ± 190.509     |
| Europe   | 15          | 1            | 3918.415 ± 217.34    | 2887.16 ± 164.53            | 2832.05 ± 151.968    |
| Europe   | 15          | 2            | 4324.825 ± 241.26    | 3209.989 ± 171.546          | 3083.75 ± 153.208    |
| Europe   | 15          | 3            | 4780.595 ± 275.929   | 3567.486 ± 199.868          | 3385.05 ± 171.432    |
| Europe   | 20          | 1            | 4066.235 ± 222.875   | 2991.726 ± 158.416          | 2953.75 ± 152.685    |
| Europe   | 20          | 2            | 4563.99 ± 229.509    | 3371.71 ± 158.704           | 3272.95 ± 149.314    |
| Europe   | 20          | 3            | 5075.63 ± 277.376    | 3764.744 ± 184.005          | 3584.95 ± 171.283    |


In [ ]:
| Topology | No. clients | No. vehicles | R/T e. [\%]       | R/G e. [\%]       | G/T e. [\%]       | R/T p. | R/G p. | G/T p. |
|----------|-------------|--------------|-------------------|-------------------|-------------------|--------|--------|--------|
| Krakow   | 10          | 1            | 52.73 ± 15.73      | 35.00 ± 14.45      | 39.54 ± 13.54      | 4      | 2      | 1      |
| Krakow   | 10          | 2            | 50.83 ± 12.69      | 44.17 ± 9.72       | 41.25 ± 8.13       | 2      | 1      | 0      |
| Krakow   | 10          | 3            | 53.85 ± 9.11       | 39.62 ± 9.75       | 46.54 ± 9.56       | 0      | 0      | 1      |
| Krakow   | 15          | 1            | 40.00 ± 17.75      | 30.31 ± 10.68      | 31.43 ± 10.83      | 4      | 0      | 0      |
| Krakow   | 15          | 2            | 30.30 ± 11.42      | 30.96 ± 8.07       | 32.48 ± 7.08       | 1      | 0      | 0      |
| Krakow   | 15          | 3            | 44.50 ± 11.30      | 33.79 ± 7.55       | 37.67 ± 6.84       | 2      | 0      | 0      |
| Krakow   | 20          | 1            | 40.35 ± 15.49      | 39.99 ± 10.38      | 28.24 ± 8.96       | 0      | 2      | 0      |
| Krakow   | 20          | 2            | 34.41 ± 10.12      | 21.82 ± 5.91       | 27.15 ± 6.14       | 0      | 0      | 0      |
| Krakow   | 20          | 3            | 27.59 ± 7.14       | 29.88 ± 7.23       | 27.39 ± 6.04       | 0      | 0      | 0      |
| Europe   | 10          | 1            | 81.00 ± 15.47      | 78.23 ± 17.40      | 73.13 ± 17.62      | 16     | 13     | 11     |
| Europe   | 10          | 2            | 83.47 ± 12.55      | 80.02 ± 14.13      | 75.83 ± 13.12      | 14     | 14     | 12     |
| Europe   | 10          | 3            | 70.00 ± 14.74      | 61.15 ± 10.45      | 57.31 ± 10.89      | 7      | 5      | 4      |
| Europe   | 15          | 1            | 47.50 ± 20.00      | 50.31 ± 19.01      | 54.69 ± 19.99      | 6      | 5      | 7      |
| Europe   | 15          | 2            | 37.92 ± 14.04      | 34.60 ± 12.53      | 43.89 ± 14.79      | 3      | 2      | 2      |
| Europe   | 15          | 3            | 39.47 ± 13.86      | 39.48 ± 10.02      | 43.58 ± 11.06      | 4      | 2      | 2      |
| Europe   | 20          | 1            | 52.65 ± 19.50      | 47.40 ± 17.53      | 42.72 ± 18.06      | 5      | 2      | 2      |
| Europe   | 20          | 2            | 39.24 ± 9.49       | 26.54 ± 11.36      | 36.70 ± 11.00      | 3      | 2      | 3      |
| Europe   | 20          | 3            | 53.04 ± 15.19      | 47.36 ± 12.39      | 47.16 ± 13.59      | 2      | 2      | 2      |
